# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

df.shape

(196194, 18)

In [2]:
# Check if missing values occur in the same rows
df_clean = df.copy()

cols_with_same_missing = ['Air Temperature', 'Wet Bulb Temperature', 'Rain Intensity', 'Total Rain', 'Precipitation Type', 'Barometric Pressure', 'Heading']
missing_mask = df_clean[cols_with_same_missing].isnull()
print(f"Rows where ALL are missing: {missing_mask.all(axis=1).sum()}")
print(f"Rows where ANY are missing: {missing_mask.any(axis=1).sum()}")

Rows where ALL are missing: 0
Rows where ANY are missing: 75962


In [3]:
# Creating output/q2_cleaned_data.csv

# Handle missing values 
numeric_cols = df_clean.select_dtypes(include = np.number).columns.tolist()
# Convert Precipitation Type to categorical for better handling
df_clean['Precipitation Type'] = df_clean['Precipitation Type'].astype('category')

categorical = ['Precipitation Type']
iqr_cols = [col for col in numeric_cols if col not in categorical]

for col in iqr_cols:
    df_clean[col].fillna(df_clean[col].median(), inplace = True)

if 'Precipitation Type' in df_clean.columns:
    df_clean['Precipitation Type'].fillna(df_clean['Precipitation Type'].mode()[0], inplace = True)

df_clean = df_clean.sort_values("Measurement Timestamp")
df_clean['Measurement Timestamp'].fillna(method = 'ffill', inplace = True)
df_clean['Measurement Timestamp'].fillna(method = 'bfill', inplace = True)


object_cols = df_clean.select_dtypes(include = 'object').columns.tolist()
for col in object_cols:
    df_clean[col].fillna('Unknown', inplace = True)

# Removing impossible values first 
if 'Humidity' in df_clean.columns:
    df_clean = df_clean[(df_clean['Humidity'] >= 0) & (df_clean['Humidity'] <= 100)]

for col in ['Rain Intensity', 'Total Rain', 'Interval Rain', 'Wind Speed', 'Maximum Wind Speed', 'Solar Radiation']:
    if col in df_clean.columns:
        df_clean = df_clean[df_clean[col] >= 0]

for col in ['Wind Direction', 'Heading']:
    if col in df_clean.columns:
        df_clean = df_clean[(df_clean[col] >= 0) & (df_clean[col] <= 360)]

# Handling outliers 
for col in iqr_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_clean[col] = df_clean[col].clip(lower =lower_bound, upper = upper_bound) # outliers are capped to preserve data size

# Count duplicates and remove duplicates
duplicate_count = df_clean.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")
df_clean.drop_duplicates(inplace = True)
print(f"Duplicates removed: {duplicate_count}")

# Check row count
print(f"Row count after cleaning: {len(df_clean)}")
df_clean.to_csv('output/q2_cleaned_data.csv', index = False)












/var/folders/1t/fdhld8652kzcc785qhfcqrqh0000gn/T/ipykernel_50679/583487711.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].fillna(df_clean[col].median(), inplace = True)
/var/folders/1t/fdhld8652kzcc785qhfcqrqh0000gn/T/ipykernel_50679/583487711.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

Number of duplicate rows: 0
Duplicates removed: 0
Row count after cleaning: 182768


In [4]:
# Generating output/q2_cleaning_report.txt
missing_info = ""

for col in df.columns:
    missing_count = df[col].isnull().sum()
    if missing_count > 0:
        missing_pct = (missing_count / len(df)) * 100

        if col in iqr_cols:
            Method = "Filled with median"
            Result = "All missing values filled" 
        elif col == 'Precipitation Type':
            Method = "Filled with mode"
            Result = "All missing values filled"
        elif col == 'Measurement Timestamp':
            Method = "Forward and backward fill"
            Result = "All missing values filled"
        elif col in object_cols:
            Method = "Filled with 'Unknown'"
            Result = "All missing values filled"
        else:
            Method = "N/A"
            Result = "N/A"
        missing_info += f"- {col}: {missing_count} missing values ({missing_pct:.2f}%)\n"
        missing_info += f"  Method: {Method}\n"
        missing_info += f"  Result: {Result}\n\n"

outlier_info = ""

for col in iqr_cols:
    Q1 =  df_clean[col].quantile(0.25)
    Q3 =  df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR  
    outliers_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()

    if outliers_count > 0:
        outlier_info = f"- {col}: Detected {outliers_count} outliers using IQR method (1.5 x IQR)\n"
        outlier_info += f"  Method: Capped at bounds [Q1 - 1.5*IQR, Q3 + 1.5*IQR]\n"
        outlier_info += f"  Bounds: [{lower_bound:.2f}, {upper_bound:.2f}]\n"
        outlier_info += f"  Result: {outliers_count} values capped\n\n"

report = f"""DATA CLEANING REPORT 
====================

Rows before cleaning: {len(df)}

Missing Data Handling:
{missing_info if missing_info else "No missing values remain after cleaning."}

Outlier Handling:
{outlier_info if outlier_info else "No outliers detected."}

Duplicates Removed: 0

Data Type Conversions:
- Precipitation Type: converted to categorical type for better handling.

Rows after cleaning: {len(df_clean)}
"""

with open('output/q2_cleaning_report.txt', 'w') as f:
    f.write(report)




In [9]:
# Generate output/q2_rows_cleaned.txt
with open('output/q2_rows_cleaned.txt', 'w') as f:
    f.write(str(len(df_clean)))


---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.
